<a href="https://colab.research.google.com/github/reesha-rsh/MLb4/blob/main/Homework/HW9_Boosting_XGBoost%2C_LightGBM%2C_CatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Homework:

- use the dataset and validation approach which you used for previous homeworks
- train xgboost, lightgbm, and catboost models and find the best hyperparameters for each algorithm
- compare results between them and previously trained random forest

# Init

In [9]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import make_scorer, fbeta_score, classification_report, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import pandas as pd
import numpy as np


from google.colab import drive
drive.mount('/content/drive')

train_full = pd.read_csv("/content/drive/MyDrive/MLb4/EDA Titanic/train.csv")
test_full = pd.read_csv("/content/drive/MyDrive/MLb4/EDA Titanic/test.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
random_state = 42

In [11]:
def generate(df,age_median,fare_median):
  useless_features = ['Name','Ticket','Cabin']
  data_cleaned = df
  data_cleaned = data_cleaned.drop(columns = useless_features)

  # generate binary values using get_dummies
  data_cleaned = pd.get_dummies(data_cleaned, columns=['Sex'],prefix=["Sex"])
  data_cleaned = pd.get_dummies(data_cleaned, columns=['Embarked'],prefix=["Embarked"])

  # Check for NaN values in the DataFrame
  nan_mask = data_cleaned.isnull()
  # Count the number of NaN values in each column
  nan_count_per_column = data_cleaned.isnull().sum()

  data_cleaned['Age'] = data_cleaned['Age'].fillna(age_median)
  data_cleaned['Fare'] = data_cleaned['Fare'].fillna(fare_median)

  return data_cleaned


In [12]:
not_features = ['PassengerId', 'Survived']

In [13]:
# get medians that will fill NaNs in generate func
age_median = train_full['Age'].median()
print(age_median)
fare_median = train_full['Fare'].median()
print(fare_median)

28.0
14.4542


In [14]:
train = generate(train_full,age_median=age_median,fare_median=fare_median)
print(train)

X_train = train.drop(not_features, axis = 1)
y_train = train['Survived']

     PassengerId  Survived  Pclass   Age  SibSp  Parch     Fare  Sex_female  \
0              1         0       3  22.0      1      0   7.2500           0   
1              2         1       1  38.0      1      0  71.2833           1   
2              3         1       3  26.0      0      0   7.9250           1   
3              4         1       1  35.0      1      0  53.1000           1   
4              5         0       3  35.0      0      0   8.0500           0   
..           ...       ...     ...   ...    ...    ...      ...         ...   
886          887         0       2  27.0      0      0  13.0000           0   
887          888         1       1  19.0      0      0  30.0000           1   
888          889         0       3  28.0      1      2  23.4500           1   
889          890         1       1  26.0      0      0  30.0000           0   
890          891         0       3  32.0      0      0   7.7500           0   

     Sex_male  Embarked_C  Embarked_Q  Embarked_S  

In [15]:
test = generate(test_full,age_median=age_median,fare_median=fare_median)
X_test = test.drop('PassengerId' , axis = 1)

In [16]:
# Initialize the StratifiedKFold object
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)

# RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
parameters = {
    "max_features": [1,  3,  5,  7,  9,  10],
    "max_samples": [0.7, 0.8, 0.9],
    "max_depth": [None, 3, 5, 7, 9, 11],
    "min_samples_leaf": [1,  3,  5,  7,  9,  11],
    'criterion': ['gini'],
    'class_weight': ['balanced']
}

rfc = RandomForestClassifier(n_estimators=25, random_state=random_state, n_jobs=-1, oob_score=True)
forest_grid_search = GridSearchCV(rfc, parameters, scoring='accuracy', cv=skf)

# Perform grid search to find the best hyperparameters
forest_grid_search.fit(X_train, y_train)

# Get the best hyperparameters and the corresponding model
best_params = forest_grid_search.best_params_
best_model = forest_grid_search.best_estimator_
best_score = forest_grid_search.best_score_


# Print the best hyperparameters
print("Best Hyperparameters:", best_params)
print("Accuracy: {:.4f}".format(best_score))


Best Hyperparameters: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 9, 'max_features': 7, 'max_samples': 0.8, 'min_samples_leaf': 3}
Accuracy: 0.8451


In [ ]:
y_pred = best_model.predict(X_test)
submission_data = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': y_pred})
submission_data.to_csv("/content/drive/MyDrive/MLb4/EDA Titanic/submissionRandomForest.csv", index=False)

**SUBMISSION SCORE:** 0.76555

#XGBoost

In [17]:
import xgboost as xgb

In [20]:
xgb_train = xgb.DMatrix(X_train, y_train, feature_names=X_train.columns)

num_rounds = 10000
param = {
    #default
    "objective": "binary:logistic",
    "eta": 0.01,
    "verbosity": 0,
    "nthread": 10,
    "random_seed": random_state,
    "eval_metric": "error",
    # "feval": custom_scorer,


    # regularization parameters
    "max_leaves": 16,
    "subsample": 0.7,
    "colsample_bytree": 0.8,
    "gamma": 0,
    # "min_child_weight": 1,


    #lightgbm approach
    "tree_method": "hist",
    "grow_policy": "lossguide"
}

results = xgb.cv(param, xgb_train, num_rounds, early_stopping_rounds=20,folds=skf, verbose_eval=100)
best_score = 1-(results["test-error-mean"].min())
n_estimators  = int(len(results.index)* 1.1)
print("Accuracy on validation: {:.4f}".format(best_score))

[0]	train-error:0.17368+0.00332	test-error:0.21100+0.01610
[31]	train-error:0.13608+0.00322	test-error:0.15939+0.02009
Accuracy on validation: 0.8417


In [21]:
model = xgb.XGBClassifier(
    #default
    objective= "binary:logistic",
    eta= 0.01,
    verbosity= 0,
    nthread= 10,
    random_seed= random_state,
    eval_metric = "error",

    n_estimators = n_estimators,


    # regularization parameters
    max_leaves= 16,
    subsample= 0.7,
    colsample_bytree= 0.9,
    gamma= 0,
    min_child_weight= 5,


    #lightgbm approach
    tree_method= "hist",
    grow_policy= "lossguide"
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
submission_data = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': y_pred})
submission_data.to_csv("/content/drive/MyDrive/MLb4/EDA Titanic/submissionXgb.csv", index=False)

**SUBMISSION SCORE:** 0.77511

# LightGBM

In [23]:
import lightgbm as lgb

In [29]:
parameters = {
    #default
    "objective": "binary",
    "learning_rate": 0.01,
    "num_threads": 10,
    "metric": 'binary_error',

    "seed":  [random_state],

    #regularization
    "colsample_bytree": 0.7,
    "subsample": 0.7,
    "subsample_freq": 1,
    "min_data_in_leaf": 7,
    'force_col_wise': 'true'

}


n_rounds = 10000

lgb_train = lgb.Dataset(X_train, label=y_train, free_raw_data=False)
result = lgb.cv(parameters, lgb_train, n_rounds, folds=skf, callbacks=([lgb.early_stopping(stopping_rounds=20),lgb.log_evaluation(period=10, show_stdv=True)]), eval_train_metric=True, return_cvbooster=True)

best_score = 1 - result["valid binary_error-mean"][-1]
n_estimators  = int(result["cvbooster"].best_iteration * 1.1)
print("Accuracy on validation: {:.4f}".format(best_score))

[LightGBM] [Info] Number of positive: 273, number of negative: 439
[LightGBM] [Info] Total Bins 226
[LightGBM] [Info] Number of data points in the train set: 712, number of used features: 10
[LightGBM] [Info] Number of positive: 274, number of negative: 439
[LightGBM] [Info] Total Bins 226
[LightGBM] [Info] Number of data points in the train set: 713, number of used features: 10
[LightGBM] [Info] Number of positive: 274, number of negative: 439
[LightGBM] [Info] Total Bins 226
[LightGBM] [Info] Number of data points in the train set: 713, number of used features: 10
[LightGBM] [Info] Number of positive: 274, number of negative: 439
[LightGBM] [Info] Total Bins 226
[LightGBM] [Info] Number of data points in the train set: 713, number of used features: 10
[LightGBM] [Info] Number of positive: 273, number of negative: 440
[LightGBM] [Info] Total Bins 226
[LightGBM] [Info] Number of data points in the train set: 713, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pa

In [30]:
lgb_model = lgb.LGBMClassifier(
    objective= "binary",
    learning_rate= 0.01,
    num_threads= 10,
    metric = 'binary_error',
    seed=  random_state,
    colsample_bytree= 0.7,
    subsample= 0.7,
    subsample_freq= 1,
    min_data_in_leaf= 7,
    force_col_wise= True,
    n_estimators  = n_estimators)


lgb_model.fit(X_train, y_train)

y_pred = lgb_model.predict(X_test)
submission_data = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': y_pred})
submission_data.to_csv("/content/drive/MyDrive/MLb4/EDA Titanic/submissionLgb.csv", index=False)


[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7
[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7
[LightGBM] [Info] Number of positive: 342, number of negative: 549
[LightGBM] [Info] Total Bins 226
[LightGBM] [Info] Number of data points in the train set: 891, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383838 -> initscore=-0.473288
[LightGBM] [Info] Start training from score -0.473288
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7


**SUBMISSION SCORE:** 0.7799

# CatBoost

In [ ]:
!pip install catboost
import catboost as ctb

In [ ]:
categorical_features_indices = np.where(X_train.dtypes != float)[0]

In [ ]:

parameters = {
    "loss_function": "Logloss",
    "eval_metric": "Accuracy",
    "iterations": 10000,
    "learning_rate": 0.01,
    "random_seed": random_state,
    "od_wait": 20,
    "od_type": "Iter",
    "thread_count": 10,
    "subsample": 0.7,
    "colsample_bylevel": 0.9
}

ctb_data = ctb.Pool(X_train,y_train)
result = ctb.cv(ctb_data, parameters, folds=skf, seed=random_state, verbose_eval=20, plot=False)

best_score = result["test-Accuracy-mean"].max()
n_estimators  = int(result["test-Accuracy-mean"].values.argmax() * 1.1)
print("Accuracy on validation: {:.4f}".format(best_score))

Training on fold [0/5]
0:	learn: 0.8188202	test: 0.7932961	best: 0.7932961 (0)	total: 2.88ms	remaining: 28.8s
20:	learn: 0.8244382	test: 0.8212291	best: 0.8268156 (16)	total: 58.8ms	remaining: 27.9s

bestTest = 0.8268156425
bestIteration = 16

Training on fold [1/5]
0:	learn: 0.8246844	test: 0.7921348	best: 0.7921348 (0)	total: 2.96ms	remaining: 29.6s
20:	learn: 0.8316971	test: 0.8202247	best: 0.8202247 (15)	total: 52.8ms	remaining: 25.1s

bestTest = 0.8202247191
bestIteration = 15

Training on fold [2/5]
0:	learn: 0.8190743	test: 0.7752809	best: 0.7752809 (0)	total: 2.84ms	remaining: 28.4s
20:	learn: 0.8359046	test: 0.8033708	best: 0.8033708 (1)	total: 56.7ms	remaining: 26.9s

bestTest = 0.8033707865
bestIteration = 1

Training on fold [3/5]
0:	learn: 0.8106592	test: 0.8258427	best: 0.8258427 (0)	total: 2.71ms	remaining: 27.1s
20:	learn: 0.8246844	test: 0.8146067	best: 0.8314607 (1)	total: 57.9ms	remaining: 27.5s

bestTest = 0.8314606742
bestIteration = 1

Training on fold [4/5]
0:	le

In [ ]:
clf = ctb.CatBoostClassifier(
    loss_function = "Logloss",
    eval_metric = "Accuracy",
    iterations= 40,
    learning_rate= 0.01,
    random_seed= random_state,
    od_wait = 20,
    od_type= "Iter",
    thread_count= 10,
    subsample= 0.7,
    colsample_bylevel= 0.9)


clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

submission_data = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': y_pred})
submission_data.to_csv("/content/drive/MyDrive/MLb4/EDA Titanic/submissionCat.csv", index=False)

0:	learn: 0.8193042	total: 766us	remaining: 29.9ms
1:	learn: 0.8069585	total: 1.92ms	remaining: 36.5ms
2:	learn: 0.8148148	total: 2.97ms	remaining: 36.7ms
3:	learn: 0.8159371	total: 3.94ms	remaining: 35.4ms
4:	learn: 0.8125701	total: 4.89ms	remaining: 34.3ms
5:	learn: 0.8125701	total: 5.6ms	remaining: 31.7ms
6:	learn: 0.8103255	total: 6.87ms	remaining: 32.4ms
7:	learn: 0.8114478	total: 7.4ms	remaining: 29.6ms
8:	learn: 0.8125701	total: 8.41ms	remaining: 29ms
9:	learn: 0.8125701	total: 9.31ms	remaining: 27.9ms
10:	learn: 0.8148148	total: 10.2ms	remaining: 26.8ms
11:	learn: 0.8136925	total: 11.1ms	remaining: 26ms
12:	learn: 0.8170595	total: 12.1ms	remaining: 25.1ms
13:	learn: 0.8193042	total: 13ms	remaining: 24.2ms
14:	learn: 0.8170595	total: 13.6ms	remaining: 22.7ms
15:	learn: 0.8170595	total: 14.6ms	remaining: 21.8ms
16:	learn: 0.8193042	total: 15.4ms	remaining: 20.9ms
17:	learn: 0.8204265	total: 16.2ms	remaining: 19.8ms
18:	learn: 0.8215488	total: 17.1ms	remaining: 18.9ms
19:	learn: 0

**SUBMISSION SCORE:** 0.7799

#Conclusion


- **Random forest**
  - Submission Accuracy: 0.76555
  - Accuracy on validation: 0.8451
- **XGBoost**
  - Submission Accuracy: 0.77511
  - Accuracy on validation: 0.8417
- **LightGBM**
  - Submission Accuracy: 0.7799
  - Accuracy on validation: 0.8440
- **CatBoost**:
  - Submission Accuracy: 0.7799
  - Accuracy on validation: 0.8238

